In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , OneHotEncoder , LabelEncoder
import pickle

In [3]:
# Data Loading and Preprocessing 
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
LabelEncoderGender = LabelEncoder()
data['Gender'] = LabelEncoderGender.fit_transform(data['Gender'])
OneHotEncoderGeography = OneHotEncoder(sparse_output=False ,handle_unknown='ignore')
geo_encoded = OneHotEncoderGeography.fit_transform(data[['Geography']])
geo_df = pd.DataFrame(geo_encoded, columns=OneHotEncoder(sparse_output=False).fit(data[['Geography']]).get_feature_names_out(['Geography']))
data = pd.concat([data.drop('Geography',axis=1), geo_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [4]:
# data Splitting and Scaling
X = data.drop('EstimatedSalary', axis=1)
Y = data['EstimatedSalary']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
Scaler = StandardScaler()
X_train = Scaler.fit_transform(X_train)
X_test = Scaler.fit_transform(X_test)

In [5]:
# Saving the Encoder and Scaler
with open('LabelEncoderGender.pkl', 'wb') as f:
    pickle.dump(LabelEncoderGender,f)

with open('OneHotEncoderGeography.pkl', 'wb') as f:
    pickle.dump(OneHotEncoderGeography,f)

with open('StandardScaler.pkl', 'wb') as f:
    pickle.dump(Scaler,f)


Building An ANN Regeression model

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [7]:
# Building An ANN Regeression model
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1) # output Layer fir regeression
    ]
)
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

f:\GENERATIVE AI\3_DL_FOR_NLP\2_ANN-Regeression-Project\venv2\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime as dt
#  Setting up TensorBoard and Early Stopping
Log_dir = 'Logs/fit/' + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=Log_dir, histogram_freq=1)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [9]:
# Training the model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 13389270016.0000 - mae: 100405.4688 - val_loss: 13027707904.0000 - val_mae: 98623.2188
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 13311550464.0000 - mae: 100021.0469 - val_loss: 12869793792.0000 - val_mae: 97825.3281
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 13025612800.0000 - mae: 98601.3516 - val_loss: 12448944128.0000 - val_mae: 95698.7812
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 12434429952.0000 - mae: 95669.1172 - val_loss: 11703333888.0000 - val_mae: 91924.6719
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11508558848.0000 - mae: 91087.6016 - val_loss: 10642296832.0000 - val_mae: 86576.8672
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 10291316736.0000 - mae: 85042.7578 - val_loss: 9339606016.0000 - val_mae: 80026.6562
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8887994368.0000 - mae: 78040.8594 - val_loss: 7929065472

In [10]:
# saving the trained model
model.save('SalaryPredictionModel.keras')

In [14]:
%load_ext tensorboard
%tensorboard --logdir Logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 7132), started 0:32:08 ago. (Use '!kill 7132' to kill it.)